In [1]:
import os
import numpy as np
from scipy.interpolate import interp1d


def calculate_delta_f(phi, free):    
    # Filter data based on conditions
    A = free[phi < 0]
    B = free[(phi > 0) & (phi < 2.2)]
    
    # Calculate free energies
    fesA = -2.49 * np.logaddexp.reduce(-1 / 2.49 * A)
    fesB = -2.49 * np.logaddexp.reduce(-1 / 2.49 * B)
    return fesB - fesA

def load_data(fes_dir, method):
    file = os.path.join(fes_dir, str(seed), "fes/100.dat")

    # Load data from the file
    data = np.loadtxt(file, comments='#')
    
    with open(file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    phi_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    phi = data[:, phi_idx]
    free = data[:, free_idx]
    return phi, free

def load_interp(fes_dir, method):
    fes_file = os.path.join(fes_dir, str(seed), "fes/100.dat")
    cv_file = os.path.join(fes_dir, str(seed), "COLVAR")

    # Load data from the file
    data = np.loadtxt(fes_file, comments='#')
    
    with open(fes_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    cv_grid = data[:, cv_idx]
    free_grid = data[:, free_idx]

    # Load data from the file
    data = np.loadtxt(cv_file, comments='#')
    
    with open(cv_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    phi_idx = keys.index('phi')

    cv = data[:, cv_idx]
    phi = data[:, phi_idx]

    # 선형 보간: deep_cv 범위 밖이면 extrapolate
    fes_interp = interp1d(
        cv_grid, 
        free_grid, 
        kind='linear', 
        fill_value="extrapolate"
    )

    free = fes_interp(cv)
    return phi, free
    
# Directory containing the subdirectories

cv_name = {
    'phi': 'phi',
    'ref': 'phi',
    'CL': 'deep.node-2',
    'DeepTDA': 'deep.node-0',
    'DeepLDA': 'deep.node-0',
}
# method = 'phi'
# date = '0120114455'
# method = 'ref'
# date = '0120114606'
# method = 'CL'
# date = '0120120020'
# method = 'DeepTDA'
# date = '0120120139'
method = 'DeepLDA'
date = '0121033441'
fes_dir = f'/home/guest_sky/geodesic-interpolation-cv/simulations/aldp/{method}/log/{date}'

delta_fs = []
# Iterate through all subdirectories
for seed in range(4):
    if method in ['phi','ref']:
        phi, free = load_data(fes_dir, method)
    else:
        phi, free = load_interp(fes_dir, method)
        
    delta_f = calculate_delta_f(phi, free)
    if np.any(np.isinf(delta_f)):
        continue
    delta_fs.append(delta_f)
    print(f"{delta_f}")

# Calculate mean and standard deviation of delta F
delta_f_mean = np.mean(delta_fs)
delta_f_std = np.std(delta_fs)

# Output results
print(f"Mean: {delta_f_mean}")
print(f"Std: {delta_f_std}")


Mean: nan
Std: nan


/home/guest_sky/.conda/envs/geodesic-cv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/guest_sky/.conda/envs/geodesic-cv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/guest_sky/.conda/envs/geodesic-cv/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/guest_sky/.conda/envs/geodesic-cv/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/guest_sky/.conda/envs/geodesic-cv/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
